<a href="https://colab.research.google.com/github/rajagopal17/Text-Analysis-with-Spacy/blob/master/Spacy_Rule_Matcher_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rc('font', size=14)
sns.set(style='white')
sns.set(style='whitegrid', color_codes=True)
import csv


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import gensim,pprint
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)
from gensim.models import Phrases
#from gensim.models.word2vec import sentences
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim import corpora, models,similarities
from gensim.models import LdaModel


In [3]:
from __future__ import unicode_literals
import spacy
import re
from spacy.tokens import doc
nlp=spacy.load('en')
import en_core_web_sm
#nlp=en_core_web_md.load()
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
parser = English()
import string
punctuations=string.punctuation

In [4]:
raw_doc = pd.read_csv('/content/drive/My Drive/Python/Reviews.csv',sep=',', skiprows=1,header=None)

##Load spaCy model
Since we will not be doing any specialized tasks such as dependency parsing and named entity recognition in this exercise, these components are disabled when loading the spaCy model.

SpaCy has a sentencizer component that can be plugged into a blank pipeline.

The sentencizer pipeline simply performs tokenization and sentence boundary detection, following which lemmas can be extracted as token properties.

In [5]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe(nlp.create_pipe('sentencizer'))

In [6]:
wip_data=pd.DataFrame(columns=['s_no','comments'])
wip_data['s_no']=raw_doc[0]
wip_data['comments']=raw_doc[9]
wip_data.head()


,s_no,comments
0,1,I have bought several of the Vitality canned d...
1,2,Product arrived labeled as Jumbo Salted Peanut...
2,3,This is a confection that has been around a fe...
3,4,If you are looking for the secret ingredient i...
4,5,Great taffy at a great price. There was a wid...


##Define text cleaner
Since the news article data comes from a raw HTML dump, it is very messy and contains a host of unnecessary symbols, social media handles, URLs and other artifacts. An easy way to clean it up is to use a regex that parses only alphanumeric strings and hyphens (so as to include hyphenated words) that are between a given length (3 and 50). This filters each document down to only meaningful text for the lemmatizer.

In [7]:
wip_data['comments']=wip_data['comments'].apply(lambda x: str(x))
wip_data['comments']=wip_data['comments'].apply(lambda x: re.sub('[^A-Z 0-9 a-z-]+','',x))

## Lemmatize the text using nlp.pipe

In [8]:
%%time
lemma_list=list([token.lemma_.lower() for token in doc if token.is_alpha and token.is_stop==False] for doc in nlp.pipe(wip_data['comments'], batch_size=1000,disable=['tagger','parser','ner']))

CPU times: user 2.56 s, sys: 21.7 ms, total: 2.58 s
Wall time: 2.59 s


In [9]:
wip_data['comments']=lemma_list
wip_data.head()

,s_no,comments
0,1,"[buy, vitality, can, dog, food, product, find,..."
1,2,"[product, arrive, label, jumbo, salted, peanut..."
2,3,"[confection, century, light, pillowy, citrus, ..."
3,4,"[look, secret, ingredient, robitussin, believe..."
4,5,"[great, taffy, great, price, wide, assortment,..."


##Convert the required column into a string to feed it to the rule matcher function

In [10]:
%%time
nlp.max_length=2585990
myfile_list=raw_doc[9].apply(lambda x: str(x)).tolist()
myfile_string=(''.join(x for x in myfile_list))
myfile=nlp(myfile_string)

CPU times: user 1min 8s, sys: 6.69 s, total: 1min 15s
Wall time: 1min 15s


##Write a function to match all the sentences which have words "coffee" & 'tea'

In [11]:
%%time
from spacy.matcher import Matcher
matcher=Matcher(nlp.vocab)
coffee_pattern = [{'POS': 'NOUN', 'LOWER': 'coffee'}]
coffee_pattern2  =[{'POS': 'NOUN','LOWER':'cofee'}]
tea_pattern     = [{'POS': 'NOUN','LOWER':'tea'}]

matcher.add('matched',None, coffee_pattern)
matcher.add('matched',None,coffee_pattern2)
matcher.add('matched',None,tea_pattern)

#########################################


CPU times: user 588 µs, sys: 0 ns, total: 588 µs
Wall time: 701 µs


In [49]:

matches = matcher(myfile)
flat_match=[]
for match_id, start, end in matches:
    matched_span = myfile[start:end]
    flat_match.append(matched_span.sent)         
    #print(str(matched_span.sent))

In [50]:
flat_match

[you for the personal, incredible service!I have never been a huge coffee fan.,
 MY FRIEND NATE THE OTHER MORNING FOR COFFEE,
 IT GOES GOOD WITH O.J. AND COFFEE AND A SLICE OF TOAST AND YOUR READY,
 More importantly, the coffee is smooth; no bitter aftertaste.  ,
 I would not go out of my way to buy it againNo tea flavor at all.,
 i cannot make tea this good.  ,
 but i know i have never enjoyed tea that was this sweet without being too sweet.  ,
 I use it to make my own coffee,
 have to have my coffee blonde and sweet - but the flavored creamers are full of the bad kinds of fat, and honestly, I hate to use manufactured "food" items.  ,
 The non-instant lowfat milk is a little sweeter and tastes fresher than regular instant low-fat milk, but does not dissolve good in cold water - which is not a problem for hot coffee.  ,
 - I would not do just the heavy cream, it made the coffee too rich.,
 I have been drinking this tea for a long time now.  ,
 I drink this tea throughout the day like o

In [51]:
df=pd.DataFrame(columns=['comments'])
df['comments']=flat_match
df

,comments
0,"(you, for, the, personal, ,, incredible, servi..."
1,"(MY, FRIEND, NATE, THE, OTHER, MORNING, FOR, C..."
2,"(IT, GOES, GOOD, WITH, O.J., AND, COFFEE, AND,..."
3,"(More, importantly, ,, the, coffee, is, smooth..."
4,"(I, would, not, go, out, of, my, way, to, buy,..."
...,...
2339,"(This, tea, is, much, more, effective, than, r..."
2340,"(Valerian, is, a, very, effective, herb, if, y..."
2341,"(a, wonderful, product!<br, />I, can, drink, i..."
2342,"(again, will, I, go, back, to, sweet, tea, or,..."
